<a href="https://colab.research.google.com/github/vanditagoyal1997/depression-analysis/blob/sankhya/AU's.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%tensorflow_version 1.15.2
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from scipy.io import wavfile as wav
from scipy import signal
from scipy.fftpack import fft,fftfreq
import os

`%tensorflow_version` only switches the major version: 1.x or 2.x.
You set: `1.15.2`. This will be interpreted as: `1.x`.


TensorFlow 1.x selected.


In [ ]:
from numpy import array
import tensorflow as tf
from tensorflow.python.keras.preprocessing.text import one_hot
from keras.preprocessing.sequence import pad_sequences
from tensorflow.python.keras.models import Sequential
from tensorflow.python.keras.layers.core import Activation, Dropout, Dense
from tensorflow.python.keras.layers import Flatten, LSTM
from keras.models import Model
from sklearn.model_selection import train_test_split
from keras.utils import to_categorical
from keras.preprocessing.sequence import pad_sequences
from tensorflow.python.keras.layers import Masking
import h5py

Using TensorFlow backend.


In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
os.listdir()
os.chdir('gdrive')
os.chdir('My Drive/final year project')

In [ ]:
df_1=pd.read_csv('train_split_Depression_AVEC2017.csv')
df_11=df_1.rename(columns={'PHQ8_Binary':'PHQ_Binary'})
df_2=pd.read_csv('full_test_split.csv')
df_3=pd.read_csv('full_test_split.csv')
df_4=pd.concat([df_11,df_2,df_3]).drop_duplicates().reset_index(drop=True)

In [ ]:
df_4.columns

Index(['Participant_ID', 'PHQ_Binary', 'PHQ8_Score', 'Gender',
       'PHQ8_NoInterest', 'PHQ8_Depressed', 'PHQ8_Sleep', 'PHQ8_Tired',
       'PHQ8_Appetite', 'PHQ8_Failure', 'PHQ8_Concentrating', 'PHQ8_Moving',
       'PHQ_Score'],
      dtype='object')

In [ ]:
id=df_4['Participant_ID'].tolist()
id=np.array(id)
score=df_4['PHQ_Binary'].tolist()
score=np.array(score)
f=os.listdir("au's")
temp=[]

for i in f:
  if len(id)==0:
    break
  elif int(i[0:3]) in id:
    temp.append(i)
len(temp)

154

In [ ]:
table=[]
flag=[]
counter=1
for i in temp:
  flag=[]
  df=pd.read_table("au's/"+i,sep=",")
  t=df.iloc[:,4:18]
  # t=df[[" AU01_r"," AU04_r"," AU15_r"]]
  my_list=[]
  for index, rows in t.iterrows(): 
      # Create list for the current row 
      my_list =[rows[' AU01_r'],rows[' AU02_r'],rows[' AU04_r'],rows[' AU05_r'],rows[' AU06_r'],rows[' AU09_r'],rows[' AU10_r'],rows[' AU12_r'],
                rows[' AU14_r'],rows[' AU15_r'],rows[' AU17_r'],rows[' AU20_r'],rows[' AU25_r'],rows[' AU26_r'],] 
        
      # append the list to the final list 
      flag.append(my_list)
  counter=counter+1
  table.append(flag)
np.array(table)

padded=pad_sequences(table,padding='post')
padded.shape

(154, 58989, 14)

In [ ]:
y=df_4['PHQ_Binary'].tolist()
print(y)
Y=to_categorical(y,num_classes=2)
print(Y)
num_classes=2

In [ ]:
y=df_4['PHQ_Binary'].tolist()
print (y)
X_train, X_test, Y_train, Y_test = train_test_split(padded, y, test_size=0.1)
y_train=to_categorical(Y_train,num_classes=2)
y_test=to_categorical(Y_test,num_classes=2)

In [ ]:
mod=Sequential()
mod.add(Masking(mask_value=0, input_shape=(None, 14))) 
mod.add(LSTM(5,return_sequences=True ))
mod.add(LSTM(10,return_sequences=True))
mod.add(LSTM(5,return_sequences=True))
mod.add(LSTM(10,return_sequences=False))
mod.add(Dropout(0.2))
# mod.add(Dense(100,activation='relu'))
mod.add(Dense(10,activation='relu'))
mod.add(Dense(2,activation='softmax'))
mod.compile(optimizer='adam', loss='categorical_crossentropy',metrics=['accuracy'])

In [ ]:
mod.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
masking_3 (Masking)          (None, None, 14)          0         
_________________________________________________________________
lstm_11 (LSTM)               (None, None, 20)          2800      
_________________________________________________________________
dropout_3 (Dropout)          (None, None, 20)          0         
_________________________________________________________________
dense_6 (Dense)              (None, None, 10)          210       
_________________________________________________________________
dense_7 (Dense)              (None, None, 2)           22        
Total params: 3,032
Trainable params: 3,032
Non-trainable params: 0
_________________________________________________________________


In [ ]:
mod.fit(X_train,y_train,validation_split=0.1,epochs=2)

Train on 124 samples, validate on 14 samples
Epoch 1/2
124/124 [==============================] - 2519s 20s/sample - loss: 0.6585 - acc: 0.7500 - val_loss: 0.6954 - val_acc: 0.5714
Epoch 2/2
124/124 [==============================] - 2669s 22s/sample - loss: 0.6368 - acc: 0.7581 - val_loss: 0.6976 - val_acc: 0.5714


In [ ]:
# mod.evaluate(X_test,y_test)
mod.predict(X_test)

In [ ]:
from keras.models import load_model
from keras.utils import CustomObjectScope
from keras.initializers import glorot_uniform
import tensorflow as tf

In [ ]:
model_json = mod.to_json()
with open("au.json", "w") as json_file:
    json_file.write(model_json)
mod.save_weights("au.h5")

In [ ]:
json_file = open('au.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
loaded_model = tf.keras.models.model_from_json(loaded_model_json)
# load weights into new model
loaded_model.load_weights("au.h5")
loaded_model.compile(loss='binary_crossentropy', optimizer='rmsprop', metrics=['accuracy'])
# load_model.evaluate(X_test,y_test)
score = loaded_model.evaluate(X_test,y_test, verbose=0)
print(score)
# loaded_model.predict(X_test)